In [0]:
"""
Configure Azure Data Lake Access for Spark

This cell sets the access key required for the Spark session to connect to
the Azure Data Lake Storage Gen2 account that stores the Goodreads datasets.

It enables read and write operations via the ABFSS protocol for this session.
The configuration must be re-run after each cluster restart.

Note:
For security, storing access keys directly in code is discouraged.
Use Azure Key Vault or Databricks Secrets in production environments.
"""

spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60302363.dfs.core.windows.net",
    "8aeNipwlgfgeg1YnUzDh8PeVxg0I5MmnwgWEORAqG5WIJ4Q/XsFa5m714y55ZfAzUw3nNaEFM/e8+AStXU0APQ=="
)

In [0]:
"""
Load and Inspect Silver Layer Datasets

This cell loads the 'books' and 'authors' datasets from the Silver layer
of the Azure Data Lake Storage into Spark DataFrames. Both datasets are
stored in Parquet format to preserve schema and data types.

After loading, a sample of records and the corresponding schema for each
DataFrame are displayed to confirm successful ingestion and verify structure.
"""

#Load the books dataset from the silver layer
books = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/books/"
)
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()

+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|author_id|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+--------------------+---------+------

In [0]:
"""
Load and Inspect Raw Reviews Data

This cell reads the 'reviews' dataset from the Silver layer of the Azure Data Lake,
which contains unprocessed Goodreads review records stored in Parquet format.

A preview of the first few rows and the dataset schema is displayed to verify
successful loading and to inspect the data structure before applying cleaning
and transformation steps.
"""

from pyspark.sql.functions import col, length, trim, count, when
# Read raw (uncleaned) reviews from the silver layer
reviews = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/reviews/")
# Peek at rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()


+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
"""
Load and Profile Raw Reviews Dataset

This cell reads the unprocessed 'reviews' dataset from the Silver layer of Azure Data Lake 
and performs an initial data quality assessment. The dataset is loaded in Parquet format 
to preserve its schema and data types.

Key Steps:
    - Load the dataset into a Spark DataFrame and preview sample records.
    - Display the schema to confirm column structure.
    - Perform basic profiling to identify data quality issues, including:
        • Total row count.
        • Missing or null values in key fields (review_id, book_id, user_id, rating).
        • Empty or null entries in 'review_text'.

The output provides an overview of potential cleaning requirements before transformation.
"""

from pyspark.sql.functions import col, length, trim, count, when
# Read raw (uncleaned) reviews from the silver layer
reviews = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/reviews/")
# Peek at rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()
# Basic profiling: counts and potential issues
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) ==
"") ).count()
print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id:{null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
"""
Clean and Prepare Reviews Dataset

This cell performs data cleaning on the raw Goodreads reviews to ensure quality
and consistency before feature engineering. The cleaning steps are applied in a 
structured sequence to handle missing values, invalid ratings, and inconsistent text.

Steps Performed:
    1. Remove rows with missing identifiers (review_id, book_id, user_id).
    2. Cast 'rating' to integer and retain only valid ratings between 1 and 5.
    3. Trim whitespace in 'review_text' and filter out very short entries (<10 characters).
    4. Remove duplicate records based on 'review_id'.
    5. Select relevant columns for the cleaned dataset.

The resulting DataFrame (`reviews_clean`) contains valid, deduplicated reviews
ready for aggregation and further processing.
"""

# Clean reviews DataFrame
df = reviews

# 1️⃣ Drop rows with missing IDs
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

# 2️⃣ Keep ratings between 1 and 5
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter((col("rating_int") >= 1) & (col("rating_int") <= 5))

# 3️⃣ Trim and filter short text
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter((length(col("review_text")) >= 10))

# 4️⃣ Remove duplicates
df = df.dropDuplicates(["review_id"])

# 5️⃣ Select cleaned columns (no date_added)
reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text", 
    "date_added"
)

reviews_clean.show(5, truncate=False)
reviews_clean.printSchema()


+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
"""
Write Cleaned Reviews Dataset to Silver Layer

This cell saves the cleaned 'reviews_clean' DataFrame back to the Silver layer 
of the Azure Data Lake in Parquet format, overwriting any previous version of 
the dataset.

After writing, the data is reloaded into a verification DataFrame to:
    - Confirm the Parquet file structure and schema.
    - Display sample records for validation.
    - Report the total number of cleaned rows successfully written.

This ensures data integrity before proceeding to feature engineering or aggregation.
"""

reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/reviews/"
)

# Verify
reviews_verified = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"✅ Cleaned rows written: {reviews_verified.count()}")


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
"""
Create Integrated Gold-Level Dataset

This cell constructs the Gold-layer dataset by joining the cleaned reviews 
with their corresponding book and author information. The join operations 
combine essential attributes from the three Silver-layer sources into a 
single analytical dataset.

Join Details:
    - Inner join between 'reviews_clean' and 'books' on 'book_id' 
      ensures only reviews with matching book records are retained.
    - Left join with 'authors' on 'author_id' adds author details 
      while preserving all existing review–book pairs.

Selected Columns:
    review_id, book_id, title, author_id, author name, user_id, rating,
    review_text, language, number of votes, and date_added.

The resulting DataFrame ('gold') represents a unified, analysis-ready
dataset to be used for feature engineering and downstream modeling.
"""

from pyspark.sql.functions import col

books = spark.read.parquet("abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/books/")
authors = spark.read.parquet("abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/authors/")
reviews_clean = spark.read.parquet("abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/processed/reviews/")

gold = reviews_clean.alias("r") \
    .join(books.alias("b"), col("r.book_id") == col("b.book_id"), "inner") \
    .join(authors.alias("a"), col("b.author_id") == col("a.author_id"), "left") \
    .select(
        col("r.review_id"),
        col("r.book_id"),
        col("b.title"),
        col("a.author_id"),
        col("a.name"),
        col("r.user_id"),
        col("r.rating"),
        col("r.review_text"),
        col("b.language_code").alias("language"),
        col("a.ratings_count").alias("n_votes"),
        col("r.date_added")
    )

gold.printSchema()
gold.show(10, truncate=False)


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: string (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+----------------------------------------------------------------------------+---------+------------------+--------------------------------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
"""
Save Integrated Gold Dataset in Delta Format

This cell writes the unified 'gold' DataFrame to the Gold layer of the Azure Data Lake 
in Delta format. The Delta format is used to support efficient querying, versioning, 
and future incremental updates.

Key Parameters:
    - format("delta"): Specifies Delta Lake as the storage format.
    - mode("overwrite"): Replaces any existing dataset in the target directory.
    - option("overwriteSchema", "true"): Ensures schema consistency with the current DataFrame.

Destination:
    abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/curated_reviews/

This step finalizes the curated dataset, making it available for advanced analytics
and feature engineering within the Gold layer.
"""

gold.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("abfss://lakehouse@goodreadsreviews60302363.dfs.core.windows.net/gold/curated_reviews/")


In [0]:
"""
Preview Records from the Curated Gold Dataset

This cell executes a SQL query within the Spark session to retrieve and display 
a sample of ten records from the 'curated_reviews' Delta table stored in the Gold layer.

Purpose:
    - Validate successful table creation and data accessibility through Spark SQL.
    - Confirm that the schema and sample records match the expected structure
      of the curated dataset.

The output provides a quick verification before proceeding to analysis or feature extraction.
"""

spark.sql("SELECT * FROM curated_reviews LIMIT 10").show(truncate=False)

+--------------------------------+--------+---------------------------------------------+---------+----------------+--------------------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------